## Setting Up:

In [ ]:
# =====================================================
# --- Imports ---
# =====================================================
import os
import sys
import warnings
import string
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from cmcrameri import cm

import massbalancemachine as mbm

# Add root of repo to import local modules
sys.path.append(os.path.join(os.getcwd(), "../../"))

# Local modules
from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.config_CH import *
from scripts.nn_helpers import *
from scripts.xgb_helpers import *
from scripts.geodata import *
from scripts.NN_networks import *
from scripts.geodata_plots import *

# Jupyter conveniences
%load_ext autoreload
%autoreload 2

warnings.filterwarnings("ignore")

# =====================================================
# --- Configuration ---
# =====================================================
cfg = mbm.SwitzerlandConfig()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data import WeightedRandomSampler, SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

seed_all(cfg.seed)
print("Using seed:", cfg.seed)

if torch.cuda.is_available():
    print("CUDA is available")
    free_up_cuda()
else:
    print("CUDA is NOT available")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)

colors = get_cmap_hex(cm.batlow, 10)
color_annual = "#c51b7d"
color_winter = colors[0]

vois_climate = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
]

vois_topographical = ["aspect_sgi", "slope_sgi", "svf"]

# Read GLAMOS stake data
data_glamos = getStakesData(cfg)

# Compute padding for monthly data
months_head_pad, months_tail_pad = mbm.data_processing.utils._compute_head_tail_pads_from_df(
    data_glamos)

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Transform data to monthly format (run or load data):
paths = {
    'csv_path': cfg.dataPath + path_PMB_GLAMOS_csv,
    'era5_climate_data':
    cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data.nc',
    'geopotential_data':
    cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure.nc',
    'radiation_save_path': cfg.dataPath + path_pcsr + 'zarr/'
}
RUN = False
data_monthly = process_or_load_data(
    run_flag=RUN,
    data_glamos=data_glamos,
    paths=paths,
    cfg=cfg,
    vois_climate=vois_climate,
    vois_topographical=vois_topographical,
    output_file='CH_wgms_dataset_monthly_LSTM_gs_no_oggm.csv')

# Create DataLoader
dataloader_gl = mbm.dataloader.DataLoader(cfg,
                                          data=data_monthly,
                                          random_seed=cfg.seed,
                                          meta_data_columns=cfg.metaData)

print('Number of unique stake measurements:', dataloader_gl.data.ID.nunique())

# same for winter and annual
data_winter = dataloader_gl.data[dataloader_gl.data.PERIOD == 'winter']
data_annual = dataloader_gl.data[dataloader_gl.data.PERIOD == 'annual']

print('Number of unique stake measurements (winter):',
      data_winter.ID.nunique())
print('Number of unique stake measurements (annual):',
      data_annual.ID.nunique())

print('Ratio of winter/annual stake measurements:',
      data_winter.ID.nunique() / data_annual.ID.nunique())

## Input data:

In [ ]:
# Ensure all test glaciers exist in the dataset
existing_glaciers = set(data_monthly.GLACIER.unique())
missing_glaciers = [g for g in TEST_GLACIERS if g not in existing_glaciers]

# Define training glaciers correctly
train_glaciers = [i for i in existing_glaciers if i not in TEST_GLACIERS]

data_test = data_monthly[data_monthly.GLACIER.isin(TEST_GLACIERS)]
data_train = data_monthly[data_monthly.GLACIER.isin(train_glaciers)]
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=TEST_GLACIERS,
                                            random_state=cfg.seed)
# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']
data_test = test_set['df_X']
data_test['y'] = test_set['y']

### Feature distribution of test set:

In [ ]:
MONTHLY_COLS = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
    'pcsr',
    'ELEVATION_DIFFERENCE',
]
STATIC_COLS = ['aspect_sgi', 'slope_sgi', 'svf']

feature_columns = MONTHLY_COLS + STATIC_COLS
cfg.setFeatures(feature_columns)

In [ ]:
colors = get_cmap_hex(cm.batlow, 10)
color_dark_blue = colors[0]
custom_palette = {'Train': color_dark_blue, 'Test': '#b2182b'}

fig = plot_tsne_overlap(data_train,
                        data_test,
                        STATIC_COLS,
                        MONTHLY_COLS,
                        sublabels=("a", "b", "c"),
                        label_fmt="({})",
                        label_xy=(0.02, 0.98),
                        label_fontsize=14,
                        n_iter=1000,
                        random_state=cfg.seed,
                        custom_palette=custom_palette)
# save figure
fig.savefig('figures/paper/fig_tsne_overlap_train_test_CH.png',
            dpi=300,
            bbox_inches='tight')

## LSTM:

### Build LSTM dataloaders:

In [ ]:
seed_all(cfg.seed)

df_train = data_train.copy()
df_train['PERIOD'] = df_train['PERIOD'].str.strip().str.lower()

df_test = data_test.copy()
df_test['PERIOD'] = df_test['PERIOD'].str.strip().str.lower()

# --- build train dataset from dataframe ---
ds_train = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_train,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True,
    normalize_target=True)

ds_test = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_test,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True,
    normalize_target=True)

train_idx, val_idx = mbm.data_processing.MBSequenceDataset.split_indices(
    len(ds_train), val_ratio=0.2, seed=cfg.seed)

### Define & train model:

In [ ]:
custom_params = {
    'lr': 0.001,
    'weight_decay': 0.0001,
    'hidden_size': 128,
    'num_layers': 2,
    'dropout': 0.2,
    'head_dropout': 0.0,
    'static_layers': 2,
    'static_hidden': [128, 64],
    'static_dropout': 0.1,
    'Fm': 9,
    'Fs': 3,
    'bidirectional': False,
    'loss_name': 'neutral',
    'loss_spec': None,
    'two_heads': False
}

# --- build model, resolve loss, train, reload best ---
model_filename = f"models/lstm_model_2025-12-01_no_oggm_norm_y.pt"

# --- loaders (fit scalers on TRAIN, apply to whole ds_train) ---
seed_all(cfg.seed)
ds_train_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_train)
ds_test_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_test)

train_dl, val_dl = ds_train_copy.make_loaders(
    train_idx=train_idx,
    val_idx=val_idx,
    batch_size_train=64,
    batch_size_val=128,
    seed=cfg.seed,
    fit_and_transform=
    True,  # fit scalers on TRAIN and transform Xm/Xs/y in-place
    shuffle_train=True,
    use_weighted_sampler=True  # use weighted sampler for training
)

# --- test loader (copies TRAIN scalers into ds_test and transforms it) ---
test_dl = mbm.data_processing.MBSequenceDataset.make_test_loader(
    ds_test_copy, ds_train_copy, batch_size=128, seed=cfg.seed)

# --- build model, resolve loss, train, reload best ---
seed_all(cfg.seed)
model = mbm.models.LSTM_MB.build_model_from_params(cfg, custom_params, device)
loss_fn = mbm.models.LSTM_MB.resolve_loss_fn(custom_params)

# Evaluate on test
state = torch.load(model_filename, map_location=device)
model.load_state_dict(state)
test_metrics, test_df_preds = model.evaluate_with_preds(
    device, test_dl, ds_test_copy)
test_rmse_a, test_rmse_w = test_metrics['RMSE_annual'], test_metrics[
    'RMSE_winter']

print('Test RMSE annual: {:.3f} | winter: {:.3f}'.format(
    test_rmse_a, test_rmse_w))

## Validation OOS:
Out of sample on test set

In [ ]:
colors = get_cmap_hex(cm.batlow, 10)
color_annual = "#c51b7d"
color_winter = colors[0]

# Areas (with clariden alias fix)
gl_area = get_gl_area(cfg)
gl_area["clariden"] = gl_area["claridenL"]

In [ ]:
scores_annual, scores_winter = compute_seasonal_scores(test_df_preds,
                                                       target_col='target',
                                                       pred_col='pred')

print("Annual scores:", scores_annual)
print("Winter scores:", scores_winter)

fig = plot_predictions_summary(
    grouped_ids=test_df_preds,
    scores_annual=scores_annual,
    scores_winter=scores_winter,
    ax_xlim=(-8, 6),
    ax_ylim=(-8, 6),
    color_annual=color_annual,
    color_winter=color_winter,
)
# save figure
fig.savefig('figures/paper/fig_predvsobs.png', dpi=300, bbox_inches='tight')

In [ ]:
gl_per_el = data_glamos[data_glamos.PERIOD == 'annual'].groupby(
    ['GLACIER'])['POINT_ELEVATION'].mean()
gl_per_el = gl_per_el.sort_values(ascending=False)

test_gl_per_el = gl_per_el[TEST_GLACIERS].sort_values().index

fig, axs = plt.subplots(2, 4, figsize=(32, 15), sharex=True)

gl_per_el = data_glamos[data_glamos.PERIOD == 'annual'].groupby(
    ['GLACIER'])['POINT_ELEVATION'].mean()
gl_per_el = gl_per_el.sort_values(ascending=False)
test_df_preds['gl_elv'] = test_df_preds['GLACIER'].map(gl_per_el)

subplot_labels = [
    '(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)', '(i)'
]

axs = PlotIndividualGlacierPredVsTruth(test_df_preds,
                                       axs=axs,
                                       subplot_labels=subplot_labels,
                                       color_annual=color_annual,
                                       color_winter=color_winter,
                                       custom_order=test_gl_per_el,
                                       gl_area=gl_area)

axs[3].set_ylabel("Modeled PMB [m w.e.]", fontsize=20)

fig.supxlabel('Observed PMB [m w.e.]', fontsize=20, y=0.06)
# two distinct handles
legend_scatter_annual = Line2D([0], [0],
                               marker='o',
                               linestyle='None',
                               linewidth=0,
                               markersize=10,
                               markerfacecolor=color_annual,
                               markeredgecolor='k',
                               markeredgewidth=0.8,
                               label='Annual')

legend_scatter_winter = Line2D([0], [0],
                               marker='o',
                               linestyle='None',
                               linewidth=0,
                               markersize=10,
                               markerfacecolor=color_winter,
                               markeredgecolor='k',
                               markeredgewidth=0.8,
                               label='Winter')

# if you already have other handles (e.g., bands/means), append these:
# handles = existing_handles + [legend_scatter_annual, legend_scatter_winter]
handles = [legend_scatter_annual, legend_scatter_winter]

# You can let matplotlib use the labels from the handles; no need to pass `labels=...`
fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=4,
           fontsize=20)

plt.subplots_adjust(hspace=0.25, wspace=0.25)
plt.show()

# save figure
fig.savefig('figures/paper/fig_predvsobs_indv.png',
            dpi=300,
            bbox_inches='tight')

## Intermediate validation OOS and IS:

In [ ]:
# Geodetic MB + per-glacier periods
geodetic_mb = get_geodetic_MB(cfg)
periods_per_glacier, geoMB_per_glacier = build_periods_per_glacier(geodetic_mb)

In [ ]:
PATH_PREDICTIONS_LSTM_OOS = os.path.join(
    cfg.dataPath, "GLAMOS", "distributed_MB_grids",
    "MBM/testing_LSTM/LSTM_no_oggm_OOS_norm_y")

PATH_PREDICTIONS_LSTM_IS = os.path.join(
    cfg.dataPath, "GLAMOS", "distributed_MB_grids",
    "MBM/testing_LSTM/LSTM_no_oggm_IS_norm_y")

In [ ]:
# Available glaciers (those with LSTM predictions)
glaciers_in_glamos = set(os.listdir(PATH_PREDICTIONS_LSTM_OOS))

# Areas (with clariden alias fix)
gl_area = get_gl_area(cfg)
gl_area["clariden"] = gl_area["claridenL"]

# Glaciers present in both geodetic periods and predictions, sorted by area (asc)
glacier_list = sorted(
    (g for g in periods_per_glacier.keys() if g in glaciers_in_glamos),
    key=lambda g: gl_area.get(g, 0))
print("Number of glaciers:", len(glacier_list))
print("Glaciers:", glacier_list)

In [ ]:
# Run comparison
ds_lstm_OS = process_geodetic_mass_balance_comparison(
    glacier_list=glacier_list,
    path_SMB_GLAMOS_csv=os.path.join(cfg.dataPath, path_SMB_GLAMOS_csv),
    periods_per_glacier=periods_per_glacier,
    geoMB_per_glacier=geoMB_per_glacier,
    gl_area=gl_area,
    test_glaciers=TEST_GLACIERS,
    path_predictions=PATH_PREDICTIONS_LSTM_OOS,
    cfg=cfg,
)

# Drop rows where any required columns are NaN
ds_lstm_OS = ds_lstm_OS.dropna(subset=['Geodetic MB', 'MBM MB'])
ds_lstm_OS = ds_lstm_OS.sort_values(by='Area')
ds_lstm_OS['GLACIER'] = ds_lstm_OS['GLACIER'].apply(lambda x: x.capitalize())

# Run comparison
ds_lstm_IS = process_geodetic_mass_balance_comparison(
    glacier_list=glacier_list,
    path_SMB_GLAMOS_csv=os.path.join(cfg.dataPath, path_SMB_GLAMOS_csv),
    periods_per_glacier=periods_per_glacier,
    geoMB_per_glacier=geoMB_per_glacier,
    gl_area=gl_area,
    test_glaciers=TEST_GLACIERS,
    path_predictions=PATH_PREDICTIONS_LSTM_IS,
    cfg=cfg,
)

# Drop rows where any required columns are NaN
ds_lstm_IS = ds_lstm_IS.dropna(subset=['Geodetic MB', 'MBM MB'])
ds_lstm_IS = ds_lstm_IS.sort_values(by='Area')
ds_lstm_IS['GLACIER'] = ds_lstm_IS['GLACIER'].apply(lambda x: x.capitalize())

### Mass balance gradients:

In [ ]:
# Stake data
# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv,
                          "CH_wgms_dataset_all.csv")
df_stakes = pd.read_csv(stake_file)

#### Gradients test:

In [ ]:
def _alpha_labels(n: int):
    """(a), (b), ... (z), (aa), (ab), ... for n>=1"""

    def to_label(k: int) -> str:
        # 0 -> a, 25 -> z, 26 -> aa, ...
        s = ""
        k += 1
        while k > 0:
            k, r = divmod(k - 1, 26)
            s = chr(97 + r) + s
        return f"({s})"

    return [to_label(i) for i in range(n)]


In [ ]:
# =====================================================
# --- Configuration ---
# =====================================================
gl_list = [
    'Plattalva',
    'Hohlaub',
    'Tsanfleuron',
    'Schwarzberg',
    'Forno',
]
ncols = len(gl_list)
cm = 1 / 2.54
fontsize = 9
tick_fontsize = 8

# =====================================================
# --- Figure + GridSpec setup ---
# =====================================================
fig = plt.figure(figsize=(28 * cm, 13 * cm), dpi=300)

# You can adjust width_ratios for column flexibility
gs = GridSpec(
    nrows=1,
    ncols=ncols,
    figure=fig,
    width_ratios=[1] * ncols,
    wspace=0.4,  # horizontal space between subplots
    hspace=0.25,  # vertical spacing (only one row here)
)

subplot_labels = _alpha_labels(ncols)

# =====================================================
# --- Panels ---
# =====================================================
for c, gl in enumerate(gl_list):
    ax = fig.add_subplot(gs[0, c])

    # --- Data loading ---
    df_lstm_a_oos, df_glamos_a_oos, df_all_a_oos = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_OOS, cfg, period="annual")
    df_lstm_w_oos, df_glamos_w_oos, df_all_w_oos = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_OOS, cfg, period="winter")

    years = df_all_w_oos.YEAR.unique()

    # --- Plotting ---
    ax = plot_lstm_by_elevation_periods(df_all_a_oos,
                                        df_all_w_oos,
                                        ax=ax,
                                        mean_linestyle='-',
                                        label_prefix='LSTM OOS',
                                        show_band=True,
                                        color_annual=color_annual,
                                        color_winter=color_winter)

    ax = plot_glamos_by_elevation_periods(df_all_a_oos,
                                          df_all_w_oos,
                                          ax=ax,
                                          show_band=False,
                                          label_prefix="GLAMOS",
                                          mean_linestyle=":",
                                          color_annual=color_annual,
                                          color_winter=color_winter)

    ax = plot_stakes_by_elevation_periods(df_stakes,
                                          gl.lower(),
                                          valid_bins=None,
                                          ax=ax,
                                          color_annual=color_annual,
                                          color_winter=color_winter,
                                          marker_size=14)

    # --- Labels, titles, style ---
    ax.set_ylabel("Elevation (m a.s.l.)" if c == 0 else "", fontsize=fontsize)
    ax.set_xlabel("")  # we use a supxlabel below

    area = gl_area.get(gl.lower(), np.nan)
    area = np.round(area, 3) if area < 0.1 else np.round(area, 1)

    ax.set_title(f"{gl} ({years.min()}–{years.max()})",
                 fontsize=fontsize,
                 pad=2)
    ax.grid(alpha=0.2)
    ax.tick_params(labelsize=tick_fontsize, pad=2)

    # --- Subplot label ---
    ax.text(
        0.02,
        0.98,
        subplot_labels[c],
        transform=ax.transAxes,
        fontsize=10,
        va="top",
        ha="left",
    )

    # --- Clean up legends per axis ---
    leg = ax.legend()
    if leg is not None:
        leg.remove()

    print(f"{gl}: {years.min()}–{years.max()}, {area} km²")

# =====================================================
# --- Shared label & legend ---
# =====================================================
fig.supxlabel("Mass balance (m w.e.)", fontsize=fontsize, y=0.05)

handles = [
    # LSTM
    Patch(facecolor=color_annual,
          alpha=0.25,
          label="MBM out-of-sample band (annual)"),
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle='-',
           label="MBM out-of-sample mean (annual)"),
    Patch(facecolor=color_winter,
          alpha=0.25,
          label="MBM out-of-sample band (winter)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle='-',
           label="MBM out-of-sample mean (winter)"),

    # GLAMOS
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (annual)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (winter)"),

    # Stakes
    Line2D([0], [0],
           marker='o',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_annual,
           markeredgewidth=1.2,
           label="Stakes mean (annual)"),
    Line2D([0], [0],
           marker='s',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_winter,
           markeredgewidth=1.2,
           label="Stakes mean (winter)"),
]

fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=4,
           fontsize=7)

plt.tight_layout()
plt.show()

# =====================================================
# --- Save ---
# =====================================================
fig.savefig("figures/paper/fig_mb_gradients_testgl_OOS.pdf",
            dpi=300,
            bbox_inches="tight")

In [ ]:
test_gl_area = {}
for x in TEST_GLACIERS:
    test_gl_area[x] = gl_area[x]
test_gl_area = dict(
    sorted(test_gl_area.items(), key=lambda item: item[1], reverse=True))
test_gl_area

## In-sample results:

### Maps: Two glaciers, two years

In [ ]:
fig, map_axes = plot_2glaciers_2years_glamos_vs_lstm(
    glacier_names=("aletsch", "rhone"),
    years_by_glacier=((2012, 2022), (2008, 2022)),
    cfg=cfg,
    df_stakes=df_stakes,
    path_distributed_mb=path_distributed_MB_glamos,
    path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
    period="annual",
    panel_label_start="a",
    apply_smoothing_fn=apply_gaussian_filter,
)

# save figure
fig.savefig('figures/paper/fig_glamos_vs_lstm_aletsch_rhone.png',
            dpi=300,
            bbox_inches='tight')

### Gradients train:

In [ ]:
# =====================================================
# --- Configuration ---
# =====================================================

gl_list = [
    'Gries',
    'Gietro',
    'Findelen',
    'Rhone',
    'Aletsch',
]

ncols = len(gl_list)
cm = 1 / 2.54
fontsize = 9
tick_fontsize = 8

# =====================================================
# --- Figure + GridSpec setup ---
# =====================================================
fig = plt.figure(figsize=(28 * cm, 15 * cm), dpi=300)

# You can adjust width_ratios for column flexibility
gs = GridSpec(
    nrows=1,
    ncols=ncols,
    figure=fig,
    width_ratios=[1] * ncols,
    wspace=0.4,  # horizontal space between subplots
    hspace=0.25,  # vertical spacing (only one row here)
)

subplot_labels = _alpha_labels(ncols)

# =====================================================
# --- Panels ---
# =====================================================
for c, gl in enumerate(gl_list):
    ax = fig.add_subplot(gs[0, c])

    # --- Data loading ---
    df_lstm_a_oos, df_glamos_a_oos, df_all_a_oos = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_OOS, cfg, period="annual")
    df_lstm_w_oos, df_glamos_w_oos, df_all_w_oos = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_OOS, cfg, period="winter")

    years = df_all_w_oos.YEAR.unique()

    # --- Plotting ---
    ax = plot_lstm_by_elevation_periods(df_all_a_oos,
                                        df_all_w_oos,
                                        ax=ax,
                                        mean_linestyle='-',
                                        label_prefix='LSTM OOS',
                                        show_band=True,
                                        color_annual=color_annual,
                                        color_winter=color_winter)

    ax = plot_glamos_by_elevation_periods(df_all_a_oos,
                                          df_all_w_oos,
                                          ax=ax,
                                          show_band=False,
                                          label_prefix="GLAMOS",
                                          mean_linestyle=":",
                                          color_annual=color_annual,
                                          color_winter=color_winter)

    ax = plot_stakes_by_elevation_periods(df_stakes,
                                          gl.lower(),
                                          valid_bins=None,
                                          ax=ax,
                                          color_annual=color_annual,
                                          color_winter=color_winter,
                                          marker_size=14)

    # --- Labels, titles, style ---
    ax.set_ylabel("Elevation (m a.s.l.)" if c == 0 else "", fontsize=fontsize)
    ax.set_xlabel("")  # we use a supxlabel below

    area = gl_area.get(gl.lower(), np.nan)
    area = np.round(area, 3) if area < 0.1 else np.round(area, 1)

    ax.set_title(f"{gl} ({years.min()}–{years.max()})",
                 fontsize=fontsize,
                 pad=2)
    ax.grid(alpha=0.2)
    ax.tick_params(labelsize=tick_fontsize, pad=2)

    # --- Subplot label ---
    ax.text(
        0.02,
        0.98,
        subplot_labels[c],
        transform=ax.transAxes,
        fontsize=10,
        va="top",
        ha="left",
    )

    # --- Clean up legends per axis ---
    leg = ax.legend()
    if leg is not None:
        leg.remove()

    print(f"{gl}: {years.min()}–{years.max()}, {area} km²")

# =====================================================
# --- Shared label & legend ---
# =====================================================
fig.supxlabel("Mass balance (m w.e.)", fontsize=fontsize, y=0.05)

handles = [
    # LSTM
    Patch(facecolor=color_annual,
          alpha=0.25,
          label="MBM out-of-sample band (annual)"),
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle='-',
           label="MBM out-of-sample mean (annual)"),
    Patch(facecolor=color_winter,
          alpha=0.25,
          label="MBM out-of-sample band (winter)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle='-',
           label="MBM out-of-sample mean (winter)"),

    # GLAMOS
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (annual)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (winter)"),

    # Stakes
    Line2D([0], [0],
           marker='o',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_annual,
           markeredgewidth=1.2,
           label="Stakes mean (annual)"),
    Line2D([0], [0],
           marker='s',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_winter,
           markeredgewidth=1.2,
           label="Stakes mean (winter)"),
]

fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=4,
           fontsize=7)

plt.tight_layout()
plt.show()

# =====================================================
# --- Save ---
# =====================================================
# save figure
fig.savefig('figures/paper/fig_mb_gradients_IS.png',
            dpi=300,
            bbox_inches='tight')

### Combined: 

In [ ]:
# =====================================================
# --- Helpers ---
# =====================================================
def pick_file_glamos(glacier,
                     year,
                     cfg,
                     path_distributed_MB_glamos,
                     period="annual"):
    """
    Find the GLAMOS grid file for a given glacier and year.
    """
    suffix = "ann" if period == "annual" else "win"
    base = os.path.join(cfg.dataPath, path_distributed_MB_glamos, "GLAMOS",
                        glacier)
    cand_lv95 = os.path.join(base, f"{year}_{suffix}_fix_lv95.grid")
    cand_lv03 = os.path.join(base, f"{year}_{suffix}_fix_lv03.grid")

    if os.path.exists(cand_lv95):
        return cand_lv95, "lv95"
    if os.path.exists(cand_lv03):
        return cand_lv03, "lv03"
    return None, None


def load_glamos_wgs84(glacier,
                      year,
                      cfg,
                      path_distributed_MB_glamos,
                      period="annual"):
    """
    Load GLAMOS mass balance data and convert it to WGS84.
    """
    path, cs = pick_file_glamos(glacier, year, cfg, path_distributed_MB_glamos,
                                period)
    if path is None:
        return None

    meta, arr = load_grid_file(path)
    da = convert_to_xarray_geodata(arr, meta)

    if cs == "lv03":
        return transform_xarray_coords_lv03_to_wgs84(da)
    if cs == "lv95":
        return transform_xarray_coords_lv95_to_wgs84(da)
    return None


def load_lstm_ds(glacier,
                 year,
                 path_pred_lstm,
                 period="annual",
                 smoothing_fn=None):
    """
    Load LSTM prediction dataset for a given glacier and year.
    Optionally apply a smoothing function to the dataset.
    """
    zpath = os.path.join(path_pred_lstm, glacier,
                         f"{glacier}_{year}_{period}.zarr")
    if not os.path.exists(zpath):
        return None

    ds = xr.open_zarr(zpath)
    if smoothing_fn is not None:
        ds = smoothing_fn(ds)
    return ds


def lonlat_names(obj):
    """
    Return longitude and latitude coordinate names for an xarray object.
    """
    coords = getattr(obj, "coords", {})
    if "lon" in coords and "lat" in coords:
        return "lon", "lat"
    if "longitude" in coords and "latitude" in coords:
        return "longitude", "latitude"
    return "lon", "lat"


def stake_overlay_rmse(ax,
                       glacier,
                       year,
                       cmap,
                       norm,
                       da_glamos,
                       ds_lstm,
                       df_stakes,
                       period="annual",
                       which="GLAMOS"):
    """
    Overlay stake data and compute RMSE between measured and modeled mass balance.

    Parameters
    ----------
    ax : matplotlib axis
        Axis on which to plot.
    glacier : str
        Glacier name.
    year : int
        Year of interest.
    cmap : colormap
        Colormap for plotting.
    norm : Normalize
        Normalization for color mapping.
    da_glamos : xarray.DataArray
        GLAMOS distributed MB data.
    ds_lstm : xarray.Dataset
        LSTM predicted dataset.
    df_stakes : pandas.DataFrame
        Stake data table with POINT_LON, POINT_LAT, and POINT_BALANCE columns.
    period : str, optional
        "annual" or "winter" period for filtering stakes.
    which : str, optional
        "GLAMOS" or "LSTM" to specify which dataset to compare to stakes.
    """
    if df_stakes is None:
        return None

    # Select glacier-year subset
    sub = df_stakes[(df_stakes.GLACIER == glacier)
                    & (df_stakes.YEAR == year)].copy()
    if period == "annual" and "PERIOD" in sub.columns:
        sub = sub[sub.PERIOD == "annual"].copy()
    if sub.empty:
        return None

    lx, ly = lonlat_names(
        ds_lstm if which == "LSTM" and ds_lstm is not None else da_glamos)

    # Safe evaluation wrappers
    def _safe_pred(ds, row):
        try:
            return get_predicted_mb(lx, ly, row, ds)
        except Exception:
            return np.nan

    def _safe_glamos(row):
        try:
            return get_predicted_mb_glamos(lx, ly, row, da_glamos)
        except Exception:
            return np.nan

    # Determine which field to compare
    if which == "GLAMOS":
        sub["FIELD"] = sub.apply(_safe_glamos, axis=1)
    else:
        sub["FIELD"] = (sub.apply(lambda r: _safe_pred(ds_lstm, r), axis=1)
                        if ds_lstm is not None else np.nan)

    hue_col = "POINT_BALANCE" if "POINT_BALANCE" in sub.columns else "FIELD"

    # Scatter overlay
    sns.scatterplot(
        data=sub,
        x="POINT_LON",
        y="POINT_LAT",
        hue=hue_col,
        palette=cmap,
        hue_norm=norm,
        ax=ax,
        s=10,
        legend=False,
    )

    # Compute RMSE
    if "POINT_BALANCE" in sub.columns and not np.all(np.isnan(sub["FIELD"])):
        return root_mean_squared_error(sub["POINT_BALANCE"], sub["FIELD"])
    return None

In [ ]:
# =====================================================
# --- Configuration ---
# =====================================================
glacier_names = ("aletsch", "rhone")
years_by_glacier = ((2008, 2022), (2008, 2022))
grad_glaciers = ["Gries", "Gietro", "Findelen", "Rhone", "Aletsch"]
period = "annual"
cm = 1 / 2.54

ylabel_size = 8
title_size = 9
label_size = 9
text_size = 8
tick_size = 8

# =====================================================
# --- Figure + Grid setup ---
# =====================================================
fig = plt.figure(figsize=(28 * cm, 33 * cm), dpi=300)

gs_global = GridSpec(2, 1, figure=fig, height_ratios=[2.2, 1], hspace=0.2)

gs_maps = gs_global[0].subgridspec(2,
                                   5,
                                   width_ratios=[1, 1, 1, 1, 0.05],
                                   wspace=0.35,
                                   hspace=0.25)

gs_grad = gs_global[1].subgridspec(1,
                                   5,
                                   width_ratios=[1, 1, 1, 1, 1],
                                   wspace=0.3)

label_iter = iter(string.ascii_lowercase)

# =====================================================
# --- ROWS 1–2: GLAMOS vs MBM maps ---
# =====================================================

for r, glacier in enumerate(glacier_names):

    # ---------------------------------------------------
    # --- 1) Compute vmin/vmax across BOTH YEARS ---
    # ---------------------------------------------------
    row_vals = []

    for year in years_by_glacier[r]:
        da_g_tmp = load_glamos_wgs84(
            glacier=glacier,
            year=year,
            cfg=cfg,
            path_distributed_MB_glamos=path_distributed_MB_glamos,
            period=period)

        ds_m_tmp = load_lstm_ds(glacier=glacier,
                                year=year,
                                path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
                                period=period,
                                smoothing_fn=apply_gaussian_filter)

        if da_g_tmp is not None:
            row_vals += [float(da_g_tmp.min()), float(da_g_tmp.max())]

        if ds_m_tmp is not None and "pred_masked" in ds_m_tmp:
            row_vals += [
                float(ds_m_tmp["pred_masked"].min()),
                float(ds_m_tmp["pred_masked"].max())
            ]

    if len(row_vals) == 0:
        continue  # no data for this glacier row

    row_vmin = max(min(row_vals), -12)
    row_vmax = max(row_vals)

    # conditional truncation
    vmax_display = min(row_vmax, 4.0)
    vmin_display = max(row_vmin, -12)

    cmap, norm = get_color_maps(row_vmin, row_vmax)

    # ---------------------------------------------------
    # --- 2) Now plot the two years ---
    # ---------------------------------------------------
    for j, year in enumerate(years_by_glacier[r]):

        # allocate axes
        if j == 0:
            ax_g = fig.add_subplot(gs_maps[r, 0])
            ax_m = fig.add_subplot(gs_maps[r, 1])
            ax_cb = None
        else:
            ax_g = fig.add_subplot(gs_maps[r, 2])
            ax_m = fig.add_subplot(gs_maps[r, 3])
            ax_cb = fig.add_subplot(gs_maps[r, 4])

        # load full datasets
        da_g = load_glamos_wgs84(
            glacier=glacier,
            year=year,
            cfg=cfg,
            path_distributed_MB_glamos=path_distributed_MB_glamos,
            period=period)

        ds_m = load_lstm_ds(glacier=glacier,
                            year=year,
                            path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
                            period=period,
                            smoothing_fn=apply_gaussian_filter)

        # ------------------------------------------
        # --- GLAMOS MAP (single imshow) ---
        # ------------------------------------------
        img_g = None
        if da_g is not None:
            img_g = da_g.plot.imshow(ax=ax_g,
                                     cmap=cmap,
                                     norm=norm,
                                     add_colorbar=False)
            ax_g.set_title(f"{glacier.capitalize()} – GLAMOS ({year})",
                           fontsize=title_size)

            mean_g = float(da_g.mean())
            var_g = float(da_g.var())

            rmse_g = stake_overlay_rmse(ax=ax_g,
                                        glacier=glacier,
                                        year=year,
                                        cmap=cmap,
                                        norm=norm,
                                        da_glamos=da_g,
                                        ds_lstm=ds_m,
                                        df_stakes=df_stakes,
                                        period=period,
                                        which="GLAMOS")

            txt = (f"RMSE: {rmse_g:.2f}\n" if rmse_g else "") + \
                  f"mean MB: {mean_g:.2f}\nvar: {var_g:.2f}"
            ax_g.text(0.04,
                      0.03,
                      txt,
                      transform=ax_g.transAxes,
                      ha="left",
                      va="bottom",
                      fontsize=text_size)
            ax_g.grid(alpha=0.2)

        # ------------------------------------------
        # --- MBM MAP (single imshow) ---
        # ------------------------------------------
        img_m = None
        if ds_m is not None and "pred_masked" in ds_m:
            img_m = ds_m["pred_masked"].plot.imshow(ax=ax_m,
                                                    cmap=cmap,
                                                    norm=norm,
                                                    add_colorbar=False)
            ax_m.set_title(f"{glacier.capitalize()} – MBM ({year})",
                           fontsize=title_size)

            mean_m = float(ds_m["pred_masked"].mean())
            var_m = float(ds_m["pred_masked"].var())

            rmse_m = stake_overlay_rmse(ax=ax_m,
                                        glacier=glacier,
                                        year=year,
                                        cmap=cmap,
                                        norm=norm,
                                        da_glamos=da_g,
                                        ds_lstm=ds_m,
                                        df_stakes=df_stakes,
                                        period=period,
                                        which="LSTM")

            txt = (f"RMSE: {rmse_m:.2f}\n" if rmse_m else "") + \
                  f"mean MB: {mean_m:.2f}\nvar: {var_m:.2f}"
            ax_m.text(0.04,
                      0.03,
                      txt,
                      transform=ax_m.transAxes,
                      ha="left",
                      va="bottom",
                      fontsize=text_size)
            ax_m.grid(alpha=0.2)

        # ------------------------------------------
        # --- Shared Colorbar on j == 1 ---
        # ------------------------------------------
        if j == 1:
            mappable = img_m if img_m is not None else img_g
            cb = fig.colorbar(mappable, cax=ax_cb)

            # apply conditional truncation:
            cb.ax.set_ylim(vmin_display, vmax_display)

            if min(row_vals) < vmin_display:
                # Coordinates in colorbar axis (0–1)
                ax_cb.plot(
                    [0.5],
                    [-0.02],  # x, y just below the bar
                    marker="v",  # downward triangle
                    color="black",
                    markersize=4,
                    clip_on=False,
                    transform=ax_cb.transAxes)

            if max(row_vals) > vmax_display:
                ax_cb.plot(
                    [0.5],
                    [1.02],  # position slightly above colorbar
                    marker="^",  # upward triangle
                    color="black",
                    markersize=4,
                    clip_on=False,
                    transform=ax_cb.transAxes)

            cb.set_label("Mass Balance [m w.e.]", fontsize=ylabel_size)
            cb.ax.tick_params(labelsize=tick_size)

        # ------------------------------------------
        # --- Axis labels & ticks ---
        # ------------------------------------------
        ax_g.set_ylabel("Latitude" if j == 0 else "", fontsize=ylabel_size)
        ax_m.set_ylabel("")

        ax_g.set_xlabel("Longitude" if r == 1 else "", fontsize=ylabel_size)
        ax_m.set_xlabel("Longitude" if r == 1 else "", fontsize=ylabel_size)

        if j > 0:
            ax_g.tick_params(labelleft=False, labelsize=tick_size)
        else:
            ax_g.tick_params(labelsize=tick_size)
        ax_m.tick_params(labelleft=False, labelsize=tick_size)

        # panel letters
        for ax in (ax_g, ax_m):
            label = next(label_iter)
            ax.text(0.04,
                    0.98,
                    f"({label})",
                    transform=ax.transAxes,
                    ha="left",
                    va="top",
                    fontsize=label_size)

# =====================================================
# --- ROW 3: Gradients (same as before) ---
# =====================================================
for c, gl in enumerate(grad_glaciers):
    ax = fig.add_subplot(gs_grad[0, c])

    df_lstm_a_is, df_glamos_a_is, df_all_a_is = \
        build_all_years_df(gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, "annual")

    df_lstm_w_is, df_glamos_w_is, df_all_w_is = \
        build_all_years_df(gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, "winter")

    # cut all to above >2000
    df_all_a_is = df_all_a_is[df_all_a_is.YEAR >= 2000]
    df_all_w_is = df_all_w_is[df_all_w_is.YEAR >= 2000]
    df_stakes = df_stakes[df_stakes.YEAR >= 2000]
    years = df_all_w_is.YEAR.unique()

    ax = plot_lstm_by_elevation_periods(df_all_a_is,
                                        df_all_w_is,
                                        ax=ax,
                                        mean_linestyle='-',
                                        label_prefix='MBM IS',
                                        show_band=True,
                                        color_annual=color_annual,
                                        color_winter=color_winter)

    ax = plot_glamos_by_elevation_periods(df_all_a_is,
                                          df_all_w_is,
                                          ax=ax,
                                          show_band=False,
                                          label_prefix="GLAMOS",
                                          mean_linestyle=":",
                                          color_annual=color_annual,
                                          color_winter=color_winter)

    ax = plot_stakes_by_elevation_periods(df_stakes,
                                          gl.lower(),
                                          valid_bins=None,
                                          ax=ax,
                                          color_annual=color_annual,
                                          color_winter=color_winter,
                                          marker_size=14)

    ax.set_title(f"{gl} ({years.min()}–{years.max()})",
                 fontsize=title_size,
                 pad=2)
    ax.grid(alpha=0.2)
    ax.tick_params(labelsize=tick_size, pad=2)
    ax.text(0.04,
            0.98,
            f"({next(label_iter)})",
            transform=ax.transAxes,
            ha="left",
            va="top",
            fontsize=label_size)
    ax.set_ylabel("Elevation (m a.s.l.)" if c == 0 else "",
                  fontsize=ylabel_size)

# =====================================================
# --- Final Layout ---
# =====================================================
plt.tight_layout()
plt.show()

fig.savefig("figures/paper/fig_combined_glamos_mbm_gradients_5glaciers.pdf",
            dpi=300,
            bbox_inches="tight")

### Geodetic MB:

In [ ]:
# Compute RMSE and Pearson correlation
rmse_nn = root_mean_squared_error(ds_lstm_IS["Geodetic MB"],
                                  ds_lstm_IS["MBM MB"])
corr_nn = np.corrcoef(ds_lstm_IS["Geodetic MB"], ds_lstm_IS["MBM MB"])[0, 1]

fig = plot_mbm_vs_geodetic_by_area_bin(
    ds_lstm_IS,
    bins=[0, 1, 5, 10, 100, np.inf],
    labels=['<1', '1-5', '5–10', '>10', '>100'],
    max_bins=4)

# save figure
fig.savefig('figures/paper/fig_geodetic_IS.pdf', dpi=300, bbox_inches='tight')

## Rest figures:

### Gradients all:

In [ ]:
nrows = 4
ncols = 5
cm = 1 / 2.54

# Create a figure with the specified number of subplots
fig, axs = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(30 * cm, 24 * cm),  # was (25, 15)
    dpi=300)

axs = axs.flatten()
gl_list = [
    'schwarzbach', 'murtel', 'plattalva', 'basodino', 'limmern', 'adler',
    'hohlaub', 'albigna', 'tsanfleuron', 'silvretta', 'gries', 'clariden',
    'gietro', 'schwarzberg', 'forno', 'allalin', 'otemma', 'findelen', 'rhone',
    'aletsch'
]
for i, gl in enumerate(gl_list):
    # Annual
    df_lstm_a, df_glamos_a, df_all_a = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="annual")

    years = df_all_a.YEAR.unique()

    # Winter
    df_lstm_w, df_glamos_w, df_all_w = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="winter")

    # if dataframe not None
    if df_all_a.empty:
        print(f"No data for glacier: {gl}")
        continue

    ax = plot_mb_by_elevation_periods_combined(df_all_a,
                                               df_all_w,
                                               df_stakes,
                                               gl.lower(),
                                               ax=axs[i])

    # area = areas_per_gl.loc[gl].Area
    axs[i].set_title(f'{gl.capitalize()} ({years.min()}-{years.max()})',
                     fontsize=fontsize,
                     pad=2)

    axs[i].grid(alpha=0.2)
    axs[i].tick_params(labelsize=6.5, pad=2)
    axs[i].set_ylabel('')
    axs[i].set_xlabel('')
    # remove legend
    axs[i].legend().remove()

axs[5].set_ylabel('Elevation (m a.s.l.)', fontsize=fontsize)

# Custom handles (bands, means, and stakes)
handles = [
    # LSTM
    Patch(facecolor=color_annual, alpha=0.25, label="LSTM band (annual)"),
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle='-',
           label="LSTM mean (annual)"),
    Patch(facecolor=color_winter, alpha=0.25, label="LSTM band (winter)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle='-',
           label="LSTM mean (winter)"),

    # GLAMOS (mean only)
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (annual)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (winter)"),

    # Stakes means
    Line2D([0], [0],
           marker='o',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_annual,
           markeredgewidth=1.2,
           label="Stakes mean (annual)"),
    Line2D([0], [0],
           marker='s',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_winter,
           markeredgewidth=1.2,
           label="Stakes mean (winter)"),
]

fig.supxlabel('Mass balance (m w.e.)', fontsize=fontsize, y=0.06)

fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=4,
           fontsize=7)

# Adjust the layout
plt.subplots_adjust(hspace=0.35, wspace=0.35)
plt.show()

# Save figure
fig.savefig('figures/paper/fig_gradients_all.pdf',
            dpi=300,
            bbox_inches='tight')

### Maps:

In [ ]:
# Example usage
GLACIER_NAME = 'aletsch'
df_lstm_two_heads_gl = ds_lstm_IS[ds_lstm_IS.GLACIER ==
                                  GLACIER_NAME.capitalize()]

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

plot_scatter_comparison(axs[0],
                        df_lstm_two_heads_gl,
                        GLACIER_NAME,
                        color_mbm=color_annual,
                        color_glamos=color_winter,
                        title_suffix="(LSTM two heads)")

# Load GLAMOS data
GLAMOS_glwmb = get_GLAMOS_glwmb(GLACIER_NAME, cfg)

MBM_glwmb_lstm = mbm_glwd_pred(PATH_PREDICTIONS_LSTM_IS, GLACIER_NAME)
MBM_glwmb_lstm.rename(columns={"MBM Balance": "MBM Balance LSTM"},
                      inplace=True)

# Merge with GLAMOS data
MBM_glwmb_lstm = MBM_glwmb_lstm.join(GLAMOS_glwmb)

# Plot the data
MBM_glwmb_lstm.plot(ax=axs[1],
                    y=['MBM Balance LSTM', 'GLAMOS Balance'],
                    marker="o",
                    color=[color_annual, color_winter])

axs[1].set_title(f"{GLACIER_NAME.capitalize()} Glacier", fontsize=24)
axs[1].set_ylabel("Mass Balance [m w.e.]", fontsize=18)
axs[1].set_xlabel("Year", fontsize=18)
axs[1].grid(True, linestyle="--", linewidth=0.5)
axs[1].legend(fontsize=14)
axs[1].set_title(f"{GLACIER_NAME.capitalize()} Glacier (LSTM)", fontsize=16)

plt.tight_layout()
plt.show()

In [ ]:
# print(MBM_glwmb_lstm.index)
# year_range = range(2008, 2025)
# for year in year_range:
#     period = 'annual'
#     fig, axes = plt.subplots(1, 2, figsize=(14, 7))
#     plot_mass_balance_comparison(
#         glacier_name=GLACIER_NAME,
#         year=year,
#         cfg=cfg,
#         df_stakes=df_stakes,
#         path_distributed_mb=path_distributed_MB_glamos,
#         path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
#         period='annual',
#         fig=fig,
#         axes=axes)
#     plt.suptitle(
#         f"{GLACIER_NAME.capitalize()} Glacier – {period.capitalize()} MB Comparison ({year})",
#         fontsize=18,
#     )
#     plt.tight_layout()
#     plt.show()

In [ ]:
year = 2016
GLACIER_NAME = 'gietro'

fig, axes = plt.subplots(1, 5, figsize=(28, 7))
plot_mass_balance_comparison(glacier_name=GLACIER_NAME,
                             year=year,
                             cfg=cfg,
                             df_stakes=df_stakes,
                             path_distributed_mb=path_distributed_MB_glamos,
                             path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
                             period='annual',
                             fig=fig,
                             axes=axes)

PATH_GLAMOS_TOPO = path_GLAMOS_topo
PATH_XR_GRIDS = os.path.join(cfg.dataPath, PATH_GLAMOS_TOPO, "xr_masked_grids")

# Load the dataset
ds = xr.open_zarr(os.path.join(PATH_XR_GRIDS, f'{GLACIER_NAME}_{year}.zarr'))

# Plot masked aspect, slope, elevation and capture returned QuadMesh objects
p_aspect = ds.masked_aspect.plot(ax=axes[2],
                                 cmap='twilight_shifted',
                                 add_colorbar=True)
p_slope = ds.masked_slope.plot(ax=axes[3], cmap='cividis', add_colorbar=True)
p_elev = ds.masked_elev.plot(ax=axes[4], cmap='terrain', add_colorbar=True)

# Customize colorbar labels
p_aspect.colorbar.set_label("Aspect (°)", fontsize=16)
p_slope.colorbar.set_label("Slope (°)", fontsize=16)
p_elev.colorbar.set_label("Elevation (m a.s.l.)", fontsize=16)

# Axis titles
axes[2].set_title("Aspect")
axes[3].set_title("Slope")
axes[4].set_title("DEM")

# --- Add panel labels (a), (b), (c), ...
panel_labels = ['(a)', '(b)', '(c)', '(d)', '(e)']
for ax, label in zip(axes, panel_labels):
    ax.text(0.02,
            0.98,
            label,
            transform=ax.transAxes,
            fontsize=18,
            va='top',
            ha='left',
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', pad=2))

plt.suptitle(f"{GLACIER_NAME.capitalize()} Glacier ({year})", fontsize=20)
plt.tight_layout()
plt.show()

# Save figure
fig.savefig('figures/paper/app_gietro_2016.png', dpi=300, bbox_inches='tight')

In [ ]:
year = 2021
GLACIER_NAME = 'schwarzberg'

fig, axes = plt.subplots(1, 5, figsize=(28, 7))
plot_mass_balance_comparison(glacier_name=GLACIER_NAME,
                             year=year,
                             cfg=cfg,
                             df_stakes=df_stakes,
                             path_distributed_mb=path_distributed_MB_glamos,
                             path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
                             period='annual',
                             fig=fig,
                             axes=axes)

PATH_GLAMOS_TOPO = path_GLAMOS_topo
PATH_XR_GRIDS = os.path.join(cfg.dataPath, PATH_GLAMOS_TOPO, "xr_masked_grids")

# Load the dataset
ds = xr.open_zarr(os.path.join(PATH_XR_GRIDS, f'{GLACIER_NAME}_{year}.zarr'))

# Plot masked aspect, slope, elevation and capture returned QuadMesh objects
p_aspect = ds.masked_aspect.plot(ax=axes[2],
                                 cmap='twilight_shifted',
                                 add_colorbar=True)
p_slope = ds.masked_slope.plot(ax=axes[3], cmap='cividis', add_colorbar=True)
p_elev = ds.masked_elev.plot(ax=axes[4], cmap='terrain', add_colorbar=True)

# Customize colorbar labels
p_aspect.colorbar.set_label("Aspect (°)", fontsize=16)
p_slope.colorbar.set_label("Slope (°)", fontsize=16)
p_elev.colorbar.set_label("Elevation (m a.s.l.)", fontsize=16)

# Axis titles
axes[2].set_title("Aspect")
axes[3].set_title("Slope")
axes[4].set_title("DEM")

# --- Add panel labels (a), (b), (c), ...
panel_labels = ['(a)', '(b)', '(c)', '(d)', '(e)']
for ax, label in zip(axes, panel_labels):
    ax.text(0.02,
            0.98,
            label,
            transform=ax.transAxes,
            fontsize=18,
            va='top',
            ha='left',
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', pad=2))

plt.suptitle(f"{GLACIER_NAME.capitalize()} Glacier ({year})", fontsize=20)
plt.tight_layout()
plt.show()

# Save figure
fig.savefig('figures/paper/app_schwarzberg_2021.png',
            dpi=300,
            bbox_inches='tight')

In [ ]:
period = 'annual'
GLACIER_NAME = 'aletsch'
year = 2001
fig, axes = plt.subplots(1, 2, figsize=(14, 7))
plot_mass_balance_comparison(glacier_name=GLACIER_NAME,
                             year=year,
                             cfg=cfg,
                             df_stakes=df_stakes,
                             path_distributed_mb=path_distributed_MB_glamos,
                             path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
                             period='annual',
                             fig=fig,
                             axes=axes)
plt.suptitle(
    f"{GLACIER_NAME.capitalize()} Glacier – {period.capitalize()} MB Comparison ({year})",
    fontsize=18,
)
plt.tight_layout()
plt.show()

In [ ]:
def plot_mbm_grids_only(
    glacier_name,
    years,
    cfg,
    path_pred_lstm,
    df_stakes,
    path_distributed_MB_glamos,
    period="annual",
    apply_smoothing=True,
    cm=1 / 2.54,
    ncols=4,
    text_size=8,
):
    """
    Plot only LSTM MB grids across years, overlay stakes, annotate RMSE/mean/var,
    and produce a *separate* truncated colorbar figure.
    """

    # ---- gather prediction datasets ----
    datasets = []
    year_list = []
    for year in years:
        zpath = os.path.join(path_pred_lstm, glacier_name,
                             f"{glacier_name}_{year}_{period}.zarr")
        if not os.path.exists(zpath):
            continue
        ds = xr.open_zarr(zpath)
        if apply_smoothing:
            ds = apply_gaussian_filter(ds)
        if "pred_masked" not in ds:
            continue
        datasets.append(ds["pred_masked"])
        year_list.append(year)

    if len(datasets) == 0:
        raise RuntimeError("No LSTM MB grids found.")

    # ---- global min/max across all maps ----
    global_vmin = min(float(da.min()) for da in datasets)
    global_vmax = max(float(da.max()) for da in datasets)
    cmap, norm = get_color_maps(global_vmin, global_vmax)

    # truncation for the colorbar only (not for the synthetic grids)
    vmin_display = max(global_vmin, -12)
    vmax_display = min(global_vmax, 5)

    # ---- plot layout ----
    n = len(datasets)

    nrows = math.ceil(n / ncols)
    fig, axes = plt.subplots(nrows,
                             ncols,
                             figsize=(ncols * 9 * cm, nrows * 6 * cm),
                             constrained_layout=True,
                             dpi=200)
    if nrows == 1:
        axes = axes.reshape(1, -1)

    mappable_for_cb = None

    # ---- per-year plot loop ----
    for idx, (da_lstm, year) in enumerate(zip(datasets, year_list)):
        r = idx // ncols
        c = idx % ncols
        ax = axes[r, c]

        # 📌 plot LSTM map normally
        mappable_for_cb = da_lstm.plot.imshow(ax=ax,
                                              cmap=cmap,
                                              norm=norm,
                                              add_colorbar=False)
        ax.set_title(f"{year}")

        # 📌 load GLAMOS + LSTM full datasets for stake evaluation (required!)
        da_glamos = load_glamos_wgs84(
            glacier=glacier_name,
            year=year,
            cfg=cfg,
            path_distributed_MB_glamos=path_distributed_MB_glamos,
            period=period)
        ds_lstm_full = load_lstm_ds(
            glacier=glacier_name,
            year=year,
            path_pred_lstm=path_pred_lstm,
            period=period,
            smoothing_fn=apply_gaussian_filter if apply_smoothing else None,
        )

        # 📌 overlay stakes + compute RMSE vs LSTM
        rmse_m = stake_overlay_rmse(
            ax=ax,
            glacier=glacier_name,
            year=year,
            cmap=cmap,
            norm=norm,
            da_glamos=da_glamos,  # <-- REQUIRED
            ds_lstm=ds_lstm_full,  # <-- REQUIRED
            df_stakes=df_stakes,
            period=period,
            which="LSTM")

        # 📌 compute field metrics for annotation
        mean_m = float(da_lstm.mean())
        var_m = float(da_lstm.var())

        annotation = ((f"RMSE: {rmse_m:.2f}\n" if rmse_m is not None else "") +
                      f"mean MB: {mean_m:.2f}\nvar: {var_m:.2f}")
        ax.text(0.04,
                0.03,
                annotation,
                transform=ax.transAxes,
                fontsize=text_size,
                ha="left",
                va="bottom")

        ax.grid(alpha=0.2)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel("")
        ax.set_ylabel("")

    # ---- turn off unused panels ----
    for j in range(idx + 1, nrows * ncols):
        axes[j // ncols, j % ncols].set_axis_off()

    # =====================================================
    # COLORBAR (separate figure)
    # =====================================================
    fig_cb = plt.figure(figsize=(2.5 * cm, 10 * cm), dpi=200)
    ax_cb = fig_cb.add_axes([0.25, 0.05, 0.35, 0.90])
    cb = fig_cb.colorbar(mappable_for_cb, cax=ax_cb)

    cb.ax.set_ylim(vmin_display, vmax_display)
    if global_vmin < vmin_display:
        ax_cb.plot([0.5], [-0.02],
                   marker="v",
                   color="black",
                   markersize=4,
                   transform=ax_cb.transAxes,
                   clip_on=False)
    if global_vmax > vmax_display:
        ax_cb.plot([0.5], [1.02],
                   marker="^",
                   color="black",
                   markersize=4,
                   transform=ax_cb.transAxes,
                   clip_on=False)

    cb.set_label("Mass Balance [m w.e.]")

    return fig, fig_cb

In [ ]:
fig_maps, fig_cb = plot_mbm_grids_only(
    glacier_name="aletsch",
    years=range(2005, 2025),
    cfg=cfg,
    path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
    df_stakes=df_stakes,
    period="annual",
    path_distributed_MB_glamos=path_distributed_MB_glamos,
    text_size=12,
)
fig_maps.savefig("figures/paper/mbm_all_aletsch_grids.png",
                 dpi=300,
                 bbox_inches="tight")
fig_cb.savefig("figures/paper//mbm_all_aletsch_colorbar.pdf",
               dpi=300,
               bbox_inches="tight")

In [ ]:
fig_maps, fig_cb = plot_mbm_grids_only(
    glacier_name="rhone",
    years=range(2007, 2025),
    cfg=cfg,
    path_pred_lstm=PATH_PREDICTIONS_LSTM_IS,
    df_stakes=df_stakes,
    period="annual",
    path_distributed_MB_glamos=path_distributed_MB_glamos,
    ncols=3,
    text_size=10,
)
fig_maps.savefig("figures/paper/mbm_all_rhone_grids.png",
                 dpi=300,
                 bbox_inches="tight")
fig_cb.savefig("figures/paper/mbm_all_rhone_colorbar.pdf",
               dpi=300,
               bbox_inches="tight")